# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [5]:
help(VOCS)

Init signature:
VOCS(
    *,
    variables: Dict[str, types.ConstrainedListValue] = {},
    constraints: Dict[str, types.ConstrainedListValue] = {},
    objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {},
    constants: Dict[str, Any] = {},
    linked_variables: Dict[str, str] = {},
) -> None
Docstring:     
Variables, Objectives, Constraints, and other Settings (VOCS) data structure
to describe optimization problems. 
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'description': 'Variables, Objectives, Constraints, and other Settings (VOCS) data structure\nto describe optimization problems. ',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'default': {},
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'default': {}, 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'additionalProperties': Fa

# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,125.310498,0.354682,1234,125.665180,124.955815,251.330360,249.911631
6,649.990617,-0.401219,1234,649.589398,650.391836,1299.178796,1300.783671
7,651.367273,0.712601,1234,652.079874,650.654673,1304.159748,1301.309345
8,759.968611,-0.018720,1234,759.949890,759.987331,1519.899781,1519.974663
9,481.955349,0.901876,1234,482.857226,481.053473,965.714451,962.106946
10,610.807590,0.640127,1234,611.447717,610.167462,1222.895434,1220.334925
11,183.144083,0.799957,1234,183.944040,182.344125,367.888079,364.688251
12,702.233836,-0.678754,1234,701.555082,702.912591,1403.110165,1405.825181
13,675.339105,-0.389846,1234,674.949259,675.728951,1349.898518,1351.457902
14,290.552929,-0.945485,1234,289.607443,291.498414,579.214887,582.996829


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-125.665180,124.955815
6,-649.589398,650.391836
7,-652.079874,650.654673
8,-759.949890,759.987331
9,-482.857226,481.053473
10,-611.447717,610.167462
11,-183.944040,182.344125
12,-701.555082,702.912591
13,-674.949259,675.728951
14,-289.607443,291.498414


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-125.665180,124.955815
6,-649.589398,650.391836
7,-652.079874,650.654673
8,-759.949890,759.987331
9,-482.857226,481.053473
10,-611.447717,610.167462
11,-183.944040,182.344125
12,-701.555082,702.912591
13,-674.949259,675.728951
14,-289.607443,291.498414


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-125.66517993,  124.95581531],
       [-649.58939779,  650.39183559],
       [-652.07987418,  650.65467254],
       [-759.94989049,  759.98733126],
       [-482.8572256 ,  481.05347305],
       [-611.44771703,  610.16746248],
       [-183.9440397 ,  182.34412543],
       [-701.55508226,  702.91259054],
       [-674.94925876,  675.72895106],
       [-289.60744348,  291.49841439]])

In [17]:
?form_constraint_data

Signature:
form_constraint_data(
    constraints: Dict,
    data: pandas.core.frame.DataFrame,
    prefix='constraint_',
)
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe)
A constraint is satisfied if the evaluation is < 0.

Args:
    constraints: Dictonary of constraints
    data: Dataframe with the data to be evaluated
    prefix: Prefix to use for the transformed data in the dataframe

Returns a dataframe with the constraint data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,249.330360,-249.911631
6,1297.178796,-1300.783671
7,1302.159748,-1301.309345
8,1517.899781,-1519.974663
9,963.714451,-962.106946
10,1220.895434,-1220.334925
11,365.888079,-364.688251
12,1401.110165,-1405.825181
13,1347.898518,-1351.457902
14,577.214887,-582.996829


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(constraints: Dict, data, prefix='feasible_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False
